<a href="https://colab.research.google.com/github/P2Enjoy/ia-web3.fr/blob/main/SpeedWatch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install all requirements (run once)

# System
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install --upgrade transformers torch accelerate bitsandbytes tensorflow onnxruntime-gpu
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Application
!pip install youtube_transcript_api==0.6.1 fastpunct nnsplit tiktoken
import nltk, os
nltk.download('punkt')

# Youtube processing

In [ ]:
#@title Put the link of the video to summarize:
video_link = "https://www.youtube.com/watch?v=jQL0ZeHtXFc" #@param {type:"string"}

In [ ]:
#@title Extract the youtube video transcript

def YTVideoToText(video_link):
  # installing & importing libraries
  from youtube_transcript_api import YouTubeTranscriptApi
  from fastpunct import FastPunct

  # fetching video transcript
  video_id = video_link.split("=")[1]
  transcript = YouTubeTranscriptApi.get_transcript(video_id)

  # iterating throughout and adding all text together (recovering ponctuations)
  flat_transcript = ""
  for tu in transcript:
      flat_transcript += ' ' + tu['text']

  return flat_transcript

video_full_text = YTVideoToText(video_link)

In [ ]:
#@title Restore video transcript punctuation

def FlatTextToPunct(video_link):
  # installing & importing libraries
  from fastpunct import FastPunct

  # The default language is 'english'
  fastpunct = FastPunct()

  return flat_transcript

video_full_text = YTVideoToText(video_link)

# Summarization

In [ ]:
#@title Load the model into memory
from transformers import pipeline

model_id = "facebook/bart-large-cnn" #@param {type:"string"}
summarizeryt = pipeline('summarization', model= model_id, device=0)

In [ ]:
from nnsplit import NNSplit
splitter = NNSplit.load("en")

# returns `Split` objects
video_sentences = splitter.split([video_full_text])[0]

for sentence in video_sentences:
  import tiktoken

  encoding = tiktoken.encoding_for_model("gpt2")
  num_tokens = len(encoding.encode(f"{sentence}"))
  print(f"{sentence}; {num_tokens}")

In [ ]:
#@title Join sentences in paragraphs of less the specified token size
summarize_token_width = 500 #@param {type:"slider", min:250, max:2048, step:250}

In [ ]:

def do_sum(sentences):
  # summarize text
  summarized_text = ''
  for sentence in sentences:
      out = summarizeryt(sentence, min_length=30, do_sample=False)
      out = out[0]
      out = out['summary_text']
      summarized_text.append(' ').append(out)

  # returning summary
  return summarized_text;

In [ ]:
#@title Show the transcript of the video
video_full_text

In [ ]:
#@title Show the summarized text of the video
sum_text